# Data Collection and Pre-processing

- Input: Raw OCR Annual Report
- Output: Sentence segmentation, Word segmentation

1. **Trích xuất và Phân đoạn**: Đọc văn bản từ file, phân tách thành câu
2. **Tách từ tiếng Việt**: Sử dụng VnCoreNLP để tách từ
3. **Làm sạch và Chuẩn hóa**: Chuyển chữ thường, loại bỏ noise, xóa stopwords

In [7]:
pip install underthesea vncorenlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.0 MB/s eta 0:00:00
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645933 sha256=ff1f5c315efd4ee99c6325c28659278d02d0942c77d0c76b68c6984d3ddceb10
  Stored in directory: /root/.cache/pip/wheels/6f/19/20/ec7083125fd06db1a19d0d3ca18806ecf4e8ed1464713b4efa
Successfully built vncorenlp


In [3]:
# Import các thư viện cần thiết
import re
import os
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

In [6]:
# Optional
base_zip_folder = '../data/raw_ocr_annual_report.zip'

if not os.path.exists('../data/raw_ocr_annual_report'):
    import zipfile
    with zipfile.ZipFile(base_zip_folder, 'r') as zip_ref:
        zip_ref.extractall('../data/')

In [8]:
banks = [
    'agribank',
    'bidv',
    'bsc',
    'mbbank',
    'msb',
    'ocb',
    'shb',
    'techcombank',
    'vib',
    'vietabank',
    'vietcombank',
    'viettinbank',
    'vpbank'
]

YEARS = range(2000, 2026)  # 2000 đến 2025
REPORT_TYPES = ['bctn', 'ptbv']
BASE_DIR = '../data/raw_ocr_annual_report'

In [9]:
from pathlib import Path
from datetime import datetime

def scan_report_files(base_dir, banks, years, report_types):
    """
    Quét tất cả file báo cáo có sẵn

    Cấu trúc thư mục:
    base_dir/
        agribank/
            bctn_2020_raw.txt
            bctn_2021_raw.txt
            ptbv_2021_raw.txt
            ...
        bidv/
            bctn_2020_raw.txt
            ...
    """
    file_list = []


    for bank in banks:
        bank_dir = Path(base_dir) / bank

        if not bank_dir.exists():
            print(f"Folder không tồn tại: {bank}/")
            continue

        bank_files = []

        for year in years:
            for report_type in report_types:
                file_name = f"{report_type}_{year}_raw.txt"
                file_path = bank_dir / file_name

                if file_path.exists():
                    file_size = file_path.stat().st_size / 1024  # KB
                    bank_files.append({
                        'bank': bank,
                        'year': year,
                        'report_type': report_type,
                        'file_path': str(file_path),
                        'file_size_kb': file_size
                    })

        if bank_files:
            file_list.extend(bank_files)
        else:
            print(f"{bank:15s} - Không có file nào")

    print(f"Tổng cộng: {len(file_list)} file")

    return pd.DataFrame(file_list)


df_files = scan_report_files(BASE_DIR, banks, YEARS, REPORT_TYPES)

Tổng cộng: 153 file


### Tách câu

In [8]:
from underthesea import sent_tokenize
from vncorenlp import VnCoreNLP
import time

In [9]:
def segment_sentences_nlp(text):
    print("phân đoạn câu bằng underthesea...")

    # Loại bỏ khoảng trắng thừa trước khi xử lý
    text = re.sub(r'\s+', ' ', text).strip()

    # Sử dụng underthesea để tách câu
    sentences = sent_tokenize(text)

    # Lọc câu quá ngắn (< 10 ký tự) - có thể là fragment hoặc noise
    filtered_sentences = []
    for sent in sentences:
        sent = sent.strip()
        if len(sent) >= 10:
            filtered_sentences.append(sent)

    print(f"Đã tách thành {len(filtered_sentences):,} câu hợp lệ")
    print(f"Đã lọc bỏ {len(sentences) - len(filtered_sentences):,} câu quá ngắn")

    return filtered_sentences

### Tách Từ trong câu

In [10]:
import os
from vncorenlp import VnCoreNLP

vncorenlp_dir = './VnCoreNLP'
if not os.path.exists(vncorenlp_dir):
    os.makedirs(vncorenlp_dir)
    print(f"Đã tạo thư mục {vncorenlp_dir}")

# Download VnCoreNLP
jar_file = os.path.join(vncorenlp_dir, 'VnCoreNLP-master/VnCoreNLP-1.2.jar')
if not os.path.exists(jar_file):
    print("Đang tải VnCoreNLP...")
    import urllib.request
    import zipfile

    # Download
    url = "https://github.com/vncorenlp/VnCoreNLP/archive/refs/heads/master.zip"
    zip_path = os.path.join(vncorenlp_dir, "VnCoreNLP.zip")
    urllib.request.urlretrieve(url, zip_path)

    # Giải nén
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(vncorenlp_dir)
else:
    print(f"VnCoreNLP đã tồn tại tại {vncorenlp_dir}")

Đã tạo thư mục ./VnCoreNLP
Đang tải VnCoreNLP...


In [11]:
def tokenize_sentences_vncorenlp(sentences):
    """
    Tách từ cho danh sách các câu sử dụng VnCoreNLP
    """
    try:
        # annotators="wseg" - chỉ sử dụng word segmentation
        rdrsegmenter = VnCoreNLP("./VnCoreNLP/VnCoreNLP-master/VnCoreNLP-1.2.jar", annotators="wseg", max_heap_size='-Xmx2g')

        tokenized_sentences = []
        total = len(sentences)

        print(f"Đang tách từ cho {total:,} câu...")

        for idx, sent in enumerate(sentences, 1):
            if idx % 100 == 0:
                print(f"  Đã xử lý: {idx}/{total} câu ({idx/total*100:.1f}%)")

            try:
                # Tách từ
                word_segmented = rdrsegmenter.tokenize(sent)

                tokenized_sent = ' '.join([' '.join(x) for x in word_segmented])
                tokenized_sentences.append(tokenized_sent)
            except Exception as e:
                print(f"  Lỗi tại câu {idx}: {str(e)[:50]}")
                tokenized_sentences.append(sent)

        rdrsegmenter.close()

        print(f"Hoàn thành tách từ cho {len(tokenized_sentences):,} câu")
        return tokenized_sentences

    except Exception as e:
        print(f"Lỗi khi khởi tạo VnCoreNLP: {e}")
        return [' '.join(re.findall(r'\w+', sent.lower())) for sent in sentences]

#### Clean data

In [12]:
minimal_stopwords = [
    'à', 'ạ', 'ừ', 'ư', 'ơ', 'ô', 'á', 'ấy', 'ầu', 'ào', 'ã', 'ả',
]

# 1. Từ phóng đại/mơ hồ: "rất", "nhiều", "cực kỳ", "đều", "cả"
# 2. Từ phủ định: "không", "chưa", "chẳng"
# 3. Thì động từ: "đã", "sẽ", "đang" (phân biệt cam kết vs thành tựu)
# 4. Mức độ cam kết: "có_thể", "nên", "cần", "phải"
# 5. So sánh: "hơn", "như", "theo", "tương_tự"
# 6. Liên từ quan trọng: "nhưng", "tuy_nhiên", "do_đó"


def clean_and_normalize_text(text, remove_stopwords=False):
    """
    Làm sạch và chuẩn hóa văn bản:

    Chỉ loại bỏ:
    - URL, email
    """
    # Chuyển về chữ thường
    text = text.lower()

    # Loại bỏ URL
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'www\.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

    # Loại bỏ email
    text = re.sub(r'\S+@\S+', '', text)

    # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()

    # Tách từ thành list
    words = text.split()

    if remove_stopwords:
        words = [word for word in words if word not in minimal_stopwords]

    # Loại bỏ từ quá ngắn (< 2 ký tự)
    words = [word for word in words if len(word) >= 2]

    return ' '.join(words)

### Xử lý file raw

In [20]:
def process_single_file(file_info, vncorenlp_segmenter):
    """
    Xử lý một file báo cáo:
    1. Đọc file
    2. Tách câu (underthesea)
    3. Tách từ (VnCoreNLP)
    4. Làm sạch
    5. Trả về DataFrame
    """
    try:
        # 1. Đọc file
        with open(file_info['file_path'], 'r', encoding='utf-8') as f:
            raw_text = f.read().strip().replace('##', '').replace('<!-- image -->', '')

            # remove all page number with format \n\n{page_num}\n\n, run loop from 1 to 300, page number like 1 or 01 if contain 1 digits
            for i in range(1, 301):
                raw_text = re.sub(r'\n\n{}\n\n'.format(i), '', raw_text)
                raw_text = re.sub(r'\n\n{}\n\n'.format(str(i).zfill(2)), '', raw_text)

        if len(raw_text) < 100:  # File quá ngắn
            return None, "File quá ngắn"

        # 2. Tách câu
        text = re.sub(r'\s+', ' ', raw_text).strip()
        sentences = segment_sentences_nlp(text)

        if len(sentences) == 0:
            return None, "Không tách được câu"

        # 3. Tách từ
        tokenized_sentences = tokenize_sentences_vncorenlp(sentences)

        # 4. Làm sạch và tạo DataFrame
        data = []
        for idx, (orig_sent, tok_sent) in enumerate(zip(sentences, tokenized_sentences)):
            cleaned = clean_and_normalize_text(tok_sent)

            if len(cleaned.split()) >= 5:
                data.append({
                    'bank': file_info['bank'],
                    'year': file_info['year'],
                    'report_type': file_info['report_type'],
                    'sentence_id': idx,
                    'original_sentence': orig_sent,
                    'tokenized_sentence': tok_sent,
                    'cleaned_sentence': cleaned,
                    'word_count': len(cleaned.split()),
                    'char_count': len(cleaned)
                })

        if len(data) == 0:
            return None, "Không có câu hợp lệ sau làm sạch"

        return pd.DataFrame(data), None

    except Exception as e:
        return None, f"Lỗi: {str(e)[:50]}"


def process_all_files(df_files):
    # Khởi tạo VnCoreNLP
    try:
        rdrsegmenter = VnCoreNLP(
            "./VnCoreNLP/VnCoreNLP-master/VnCoreNLP-1.2.jar",
            annotators="wseg",
            max_heap_size='-Xmx2g'
        )
    except Exception as e:
        print(f"Không thể khởi tạo VnCoreNLP: {e}")
        return None

    all_data = []
    success_count = 0
    fail_count = 0
    total_sentences = 0

    start_time = time.time()

    for idx, file_info in df_files.iterrows():
        progress = (idx + 1) / len(df_files) * 100

        print(f"[{idx+1}/{len(df_files)}] ({progress:.1f}%) Processing: {file_info['bank']}/{file_info['report_type']}_{file_info['year']}_raw.txt")

        df_result, error = process_single_file(file_info, rdrsegmenter)

        if df_result is not None:
            all_data.append(df_result)
            success_count += 1
            total_sentences += len(df_result)
            print(f"{len(df_result):,} câu")
        else:
            fail_count += 1
            print(f"{error}")

    # Đóng VnCoreNLP
    rdrsegmenter.close()

    # Gộp tất cả DataFrame
    if all_data:
        df_final = pd.concat(all_data, ignore_index=True)
    else:
        df_final = pd.DataFrame()

    elapsed_time = time.time() - start_time

    print(f"Thành công:          {success_count}/{len(df_files)} file")
    print(f"Thất bại:            {fail_count}/{len(df_files)} file")
    print("="*70)

    return df_final


# CHẠY XỬ LÝ
if len(df_files) > 0:
    df_all_sentences = process_all_files(df_files)
else:
    print("Không có file để xử lý!")

[1/153] (0.7%) Processing: agribank/bctn_2015_raw.txt
phân đoạn câu bằng underthesea...
Đã tách thành 331 câu hợp lệ
Đã lọc bỏ 2 câu quá ngắn
Đang tách từ cho 331 câu...
  Đã xử lý: 100/331 câu (30.2%)
  Đã xử lý: 200/331 câu (60.4%)
  Đã xử lý: 300/331 câu (90.6%)
Hoàn thành tách từ cho 331 câu
282 câu
[2/153] (1.3%) Processing: agribank/bctn_2016_raw.txt
phân đoạn câu bằng underthesea...
Đã tách thành 197 câu hợp lệ
Đã lọc bỏ 5 câu quá ngắn
Đang tách từ cho 197 câu...
  Đã xử lý: 100/197 câu (50.8%)
Hoàn thành tách từ cho 197 câu
197 câu
[3/153] (2.0%) Processing: agribank/bctn_2017_raw.txt
phân đoạn câu bằng underthesea...
Đã tách thành 254 câu hợp lệ
Đã lọc bỏ 0 câu quá ngắn
Đang tách từ cho 254 câu...
  Đã xử lý: 100/254 câu (39.4%)
  Đã xử lý: 200/254 câu (78.7%)
Hoàn thành tách từ cho 254 câu
249 câu
[4/153] (2.6%) Processing: agribank/bctn_2018_raw.txt
phân đoạn câu bằng underthesea...
Đã tách thành 303 câu hợp lệ
Đã lọc bỏ 0 câu quá ngắn
Đang tách từ cho 303 câu...
  Đã xử lý:

### Lưu kết quả

In [ ]:
if df_all_sentences is not None and len(df_all_sentences) > 0:
    output_csv = '../data/all_banks_preprocessed_sentences.csv'
    df_all_sentences.to_csv(output_csv, index=False, encoding='utf-8')